In [1]:
import os

In [2]:
import pandas as pd

In [3]:
from IPython.display import display, Markdown

In [4]:
def md(x):
     display(Markdown(x))

In [5]:
dataset_2022_url = 'https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2022.zip'

In [6]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/600.1.17\
 (KHTML, like Gecko) Version/8.0 Safari/600.1.17'

In [7]:
os.chdir(r'C:\Users\rgarcia\Documents\data') ; os.getcwd()

'C:\\Users\\rgarcia\\Documents\\data'

In [8]:
os.system(f'wget --header="User-Agent: {user_agent}" {dataset_2022_url}')

0

In [9]:
os.system('unzip -p consulta_cand_2022.zip consulta_cand_2022_BRASIL.csv > consulta_cand_2022_BRASIL.csv')

0

In [10]:
os.system(f'del consulta_cand_2022.zip')

0

In [11]:
os.chdir(r'C:\Users\rgarcia\Documents\code') ; print(os.getcwd())

C:\Users\rgarcia\Documents\code


In [12]:
dataset_2018 = r'C:\Users\rgarcia\Documents\Data\consulta_cand_2018_BRASIL.csv'
dataset_2022 = r'C:\Users\rgarcia\Documents\Data\consulta_cand_2022_BRASIL.csv'

In [13]:
cand_2018 = pd.read_csv(dataset_2018, sep=';', encoding='ISO-8859-1', decimal=',', low_memory=False)
cand_2022 = pd.read_csv(dataset_2022, sep=';', encoding='ISO-8859-1', decimal=',', low_memory=False)

## Histórico

In [14]:
md(f'> ### Pedidos de registro de candidatura em 2022: \n > # {cand_2022.shape[0]}')

> ### Pedidos de registro de candidatura em 2022: 
 > # 29086

In [15]:
historico = pd.DataFrame({'Eleição':[1994, 1998, 2002, 2006, 2010, 2014, 2018, 2022],
             'Candidatos':[6623, 15040, 18049, 19263, 22537, 26161, 29085, cand_2022.shape[0]]})

historico['Variação (%)'] = (historico['Candidatos'].pct_change()*100).round(2)

# Histórico de número de candidatos

In [16]:
historico

,Eleição,Candidatos,Variação (%)
0,1994,6623,NaN
1,1998,15040,127.09
2,2002,18049,20.01
3,2006,19263,6.73
4,2010,22537,17.00
5,2014,26161,16.08
6,2018,29085,11.18
7,2022,29086,0.00


## Cenário por cargo disputado

In [17]:
cand_cargo_2018 = cand_2018.groupby(['DS_CARGO']).SQ_CANDIDATO.count().reset_index()

In [18]:
cand_cargo_2022 = cand_2022.groupby(['DS_CARGO']).SQ_CANDIDATO.count().reset_index()

In [19]:
cand_cargo_2022['ANO_ELEICAO'] = 2022

In [20]:
cand_cargo_2018['ANO_ELEICAO'] = 2018

In [21]:
df = pd.concat([cand_cargo_2018, cand_cargo_2022]).reset_index(drop=True)

In [22]:
df = df.pivot(columns='ANO_ELEICAO', index='DS_CARGO', values='SQ_CANDIDATO').reset_index()

In [23]:
df['Variação'] = ((df[2022]/df[2018]-1)*100).round(1)

In [24]:
df = df.reset_index(drop=True)

In [25]:
df.index.names = ['']

In [26]:
df.columns.name = ''

In [27]:
df.columns = ['Cargo disputado', '2018', '2022', 'Variação (%)']

In [28]:
df['Cargo disputado'] = df['Cargo disputado'].str.title()

In [29]:
df['Direção'] = df['Variação (%)'].apply(lambda x: 'Aumento' if x > 0 else 'Queda')

# Número de candidatos por cargo

In [30]:
vagas = pd.DataFrame({'vagas_2018': [54, 54, 24, 1035, 513, 27, 1, 54, 27, 1],
                      'vagas_2022': [27, 27, 24, 1035, 513, 27, 1, 27, 27, 1]})

In [31]:
df = pd.concat([df,vagas], axis=1)

In [32]:
df['concorrencia_2018'] = (df['2018']/df.vagas_2018).round(1)

In [33]:
df['concorrencia_2022'] = (df['2022']/df.vagas_2022).round(1)

In [34]:
df['concorrencia_chg'] = (((df.concorrencia_2022/df.concorrencia_2018)-1)*100).round(1)

In [35]:
df

,Cargo disputado,2018,2022,Variação (%),Direção,vagas_2018,vagas_2022,concorrencia_2018,concorrencia_2022,concorrencia_chg
0,1º Suplente,398,259,-34.9,Queda,54,27,7.4,9.6,29.7
1,2º Suplente,405,262,-35.3,Queda,54,27,7.5,9.7,29.3
2,Deputado Distrital,981,606,-38.2,Queda,24,24,40.9,25.2,-38.4
3,Deputado Estadual,17941,16668,-7.1,Queda,1035,1035,17.3,16.1,-6.9
4,Deputado Federal,8588,10569,23.1,Aumento,513,513,16.7,20.6,23.4
5,Governador,230,223,-3.0,Queda,27,27,8.5,8.3,-2.4
6,Presidente,16,13,-18.8,Queda,1,1,16.0,13.0,-18.8
7,Senador,369,240,-35.0,Queda,54,27,6.8,8.9,30.9
8,Vice-Governador,236,233,-1.3,Queda,27,27,8.7,8.6,-1.1
9,Vice-Presidente,16,13,-18.8,Queda,1,1,16.0,13.0,-18.8


In [36]:
custom_dict = {'Presidente':0, 'Vice-Presidente': 1, 'Governador':2, 'Vice-Governador':3, 
               'Senador':4, '1º Suplente':5, '2º Suplente':6, 'Deputado Federal':7, 'Deputado Estadual':8, 'Deputado Distrital':9}  
df['Ordem'] = df['Cargo disputado'].map(custom_dict)
df = df[df['Ordem'].isin([0,2,4,7,8,9])].sort_values(by='Ordem')
df = df[['Cargo disputado', '2018', '2022', 'concorrencia_2018', 'concorrencia_2022', 'concorrencia_chg']]
df.columns = ['Cargo disputado', 'Candidatos 2018', 'Candidatos 2022',
              'Concorrência* 2018', 'Concorrência* 2022', 'Variação (%)']
df

,Cargo disputado,Candidatos 2018,Candidatos 2022,Concorrência* 2018,Concorrência* 2022,Variação (%)
6,Presidente,16,13,16.0,13.0,-18.8
5,Governador,230,223,8.5,8.3,-2.4
7,Senador,369,240,6.8,8.9,30.9
4,Deputado Federal,8588,10569,16.7,20.6,23.4
3,Deputado Estadual,17941,16668,17.3,16.1,-6.9
2,Deputado Distrital,981,606,40.9,25.2,-38.4


In [37]:
cargo = df

## Deputado estadual por UF

In [38]:
dep_est_2018 = cand_2018[cand_2018.DS_CARGO == 'DEPUTADO ESTADUAL'].groupby(['SG_UF', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_est_2022 = cand_2022[cand_2022.DS_CARGO == 'DEPUTADO ESTADUAL'].groupby(['SG_UF', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_est_uf = pd.concat([dep_est_2018, dep_est_2022]).pivot(index='SG_UF', columns='ANO_ELEICAO', values= 'SQ_CANDIDATO').reset_index()

dep_est_uf.columns.names = ['']

dep_est_uf['Variação (%)'] = ((dep_est_uf[2022]/dep_est_uf[2018]-1)*100).round(1)

dep_est_uf.columns.names = ['Deputado estadual']

dep_est_uf = dep_est_uf.sort_values(by='Variação (%)', ascending=False).reset_index(drop=True)

In [39]:
dep_est_uf.columns = ['Partido', 'Eleição 2018', 'Eleição 2022', 'Variação (%)']
dep_est_uf

,Partido,Eleição 2018,Eleição 2022,Variação (%)
0,BA,643,910,41.5
1,TO,234,311,32.9
2,SC,469,613,30.7
3,PR,767,898,17.1
4,MS,355,397,11.8
5,PB,429,464,8.2
6,MA,535,558,4.3
7,MG,1382,1407,1.8
8,RS,853,828,-2.9
9,RN,330,320,-3.0


## Deputado federal por UF

In [40]:
dep_fed_2018 = cand_2018[cand_2018.DS_CARGO == 'DEPUTADO FEDERAL'].groupby(['SG_UF', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_fed_2022 = cand_2022[cand_2022.DS_CARGO == 'DEPUTADO FEDERAL'].groupby(['SG_UF', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_fed_uf = pd.concat([dep_fed_2018, dep_fed_2022]).pivot(index='SG_UF', columns='ANO_ELEICAO', values= 'SQ_CANDIDATO').reset_index()

dep_fed_uf['Variação (%)'] = ((dep_fed_uf[2022]/dep_fed_uf[2018]-1)*100).round(1)

dep_fed_uf.columns.names = ['Deputado Federal']

dep_fed_uf = dep_fed_uf.sort_values(by='Variação (%)', ascending=False).reset_index(drop=True)



In [41]:
dep_fed_uf.columns = ['Partido', 'Eleição 2018', 'Eleição 2022', 'Variação (%)']
dep_fed_uf

,Partido,Eleição 2018,Eleição 2022,Variação (%)
0,PA,141,322,128.4
1,AL,83,181,118.1
2,TO,88,160,81.8
3,MA,213,365,71.4
4,GO,229,389,69.9
5,AC,84,140,66.7
6,CE,265,414,56.2
7,RN,121,186,53.7
8,PB,159,244,53.5
9,BA,503,771,53.3


In [42]:
count = 1

for i in [historico, df, dep_est_uf, dep_fed_uf]:

    i.to_csv(f'tabela_{count}.csv', index=False)
    count += 1

In [43]:
nome_partido = {'PR':'PL', 'PPS':'CIDADANIA', 'PRB':'REPUBLICANOS', 'DEM':'UNIÃO (DEM+PSL)', 'PTC':'AGIR', 'PSL':'UNIÃO (DEM+PSL)',
                'PPL': 'PC do B', 'PRP':'PATRIOTA', 'PHS':'PODE'}

In [44]:
cand_2018.SG_PARTIDO = cand_2018.SG_PARTIDO.replace(nome_partido)

In [45]:
cand_2022.SG_PARTIDO = cand_2022.SG_PARTIDO.replace('UNIÃO','UNIÃO (DEM+PSL)')

In [46]:
dep_est_partido_2022 = cand_2022[cand_2022['DS_CARGO'] == 'DEPUTADO ESTADUAL'].groupby(['SG_PARTIDO', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_est_partido_2018 = cand_2018[cand_2018['DS_CARGO'] == 'DEPUTADO ESTADUAL'].groupby(['SG_PARTIDO', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_est_compara = pd.concat([dep_est_partido_2018, 
           dep_est_partido_2022]).pivot(columns='ANO_ELEICAO', 
                                                              index='SG_PARTIDO', values='SQ_CANDIDATO').fillna(0).astype(int)

dep_est_compara['Variação (%)'] = (((dep_est_compara[2022]/dep_est_compara[2018])-1)*100).round(1)

dep_est_compara = dep_est_compara.sort_values(by='Variação (%)', ascending=False)

In [47]:
dep_fed_partido_2022 = cand_2022[cand_2022['DS_CARGO'] == 'DEPUTADO FEDERAL'].groupby(['SG_PARTIDO', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_fed_partido_2018 = cand_2018[cand_2018['DS_CARGO'] == 'DEPUTADO FEDERAL'].groupby(['SG_PARTIDO', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_fed_compara = pd.concat([dep_fed_partido_2018, 
           dep_fed_partido_2022]).pivot(columns='ANO_ELEICAO', 
                                                              index='SG_PARTIDO', values='SQ_CANDIDATO').fillna(0).astype(int)

dep_fed_compara['Variação (%)'] = (((dep_fed_compara[2022]/dep_fed_compara[2018])-1)*100).round(1)

dep_fed_compara = dep_fed_compara.sort_values(by='Variação (%)', ascending=False)

In [48]:
dep_est_compara.columns.name = "Deputado Estadual"

In [49]:
dep_fed_compara.columns.name = "Deputado Federal"

In [50]:
dep_est_compara = dep_est_compara.reset_index()

In [51]:
dep_fed_compara = dep_fed_compara.reset_index()

In [52]:
dep_est_compara.columns = ['Partido', 'Eleição 2018', 'Eleição 2022', 'Variação (%)']
dep_fed_compara.columns = ['Partido', 'Eleição 2018', 'Eleição 2022', 'Variação (%)']

In [53]:
dep_est_compara

,Partido,Eleição 2018,Eleição 2022,Variação (%)
0,UP,0,19,inf
1,PL,472,995,110.8
2,PMB,297,566,90.6
3,PTB,424,768,81.1
4,NOVO,125,212,69.6
5,REPUBLICANOS,533,873,63.8
6,PSD,435,690,58.6
7,PP,524,778,48.5
8,PDT,572,792,38.5
9,MDB,636,831,30.7


In [54]:
dep_fed_compara

,Partido,Eleição 2018,Eleição 2022,Variação (%)
0,UP,0,18,inf
1,PTB,144,507,252.1
2,PP,151,509,237.1
3,PL,164,507,209.1
4,SOLIDARIEDADE,163,408,150.3
5,PSC,189,429,127.0
6,PSD,208,427,105.3
7,REPUBLICANOS,259,529,104.2
8,PMB,117,228,94.9
9,PCO,31,57,83.9


In [55]:
dep_est_partido = dep_est_compara; dep_fed_partido = dep_fed_compara

In [56]:
df_export = ['historico', 'cargo', 'dep_est_uf', 'dep_fed_uf', 'dep_est_partido', 'dep_fed_partido']

In [57]:
os.chdir(r'C:\Users\rgarcia\Documents\data')

In [58]:
for i in df_export:
    exec(f'{i}.to_csv("{i}.csv", index=False)')

In [59]:
os.chdir(r'C:\Users\rgarcia\Documents\code')

In [61]:
os.listdir(r'C:\Users\rgarcia\Documents\code')

['.ipynb_checkpoints',
 'Candidatos_dimensao.ipynb',
 'Candidatos_medicos.ipynb',
 'consulta_cand_2022.zip',
 'GFED4.1s_2022_beta.hdf5',
 'mudancas_saude.csv',
 'nomes_saude.csv',
 'nomes_saude_2022.csv',
 'Número candidatos.ipynb',
 'ocupacao_saude.csv',
 'ocupacao_saude_2022.csv',
 'piramide_etaria.csv',
 'pydraft.py',
 'tabela_1.csv',
 'tabela_2.csv',
 'tabela_3.csv',
 'tabela_4.csv',
 'tabela_5.csv',
 'tabela_6.csv',
 'tabela_7.csv']